In [26]:
import pickle
import pandas as pd

In [27]:
with open('model.bin', 'rb') as f_in:
    dv, model = pickle.load(f_in)

/Users/stepan_paraska/Library/Caches/pypoetry/virtualenvs/mlflow-course-m_1PGH2y-py3.12/lib/python3.12/site-packages/sklearn/base.py:380: InconsistentVersionWarning: Trying to unpickle estimator DictVectorizer from version 1.5.0 when using version 1.6.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/Users/stepan_paraska/Library/Caches/pypoetry/virtualenvs/mlflow-course-m_1PGH2y-py3.12/lib/python3.12/site-packages/sklearn/base.py:380: InconsistentVersionWarning: Trying to unpickle estimator LinearRegression from version 1.5.0 when using version 1.6.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [28]:
categorical = ['PULocationID', 'DOLocationID']

def read_data(year: int, month: int):
    filename = f'https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_{year:04}-{month:02}.parquet'
    df = pd.read_parquet(filename)
    
    df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
    df['duration'] = df.duration.dt.total_seconds() / 60

    df = df[(df.duration >= 1) & (df.duration <= 60)].copy()

    df[categorical] = df[categorical].fillna(-1).astype('int').astype('str')

    return df

In [32]:
year = 2023
month = 3

df = read_data(year=year, month=month)
df['ride_id'] = f'{year:04d}/{month:02d}_' + df.index.astype('str')

In [33]:
dicts = df[categorical].to_dict(orient='records')
X_val = dv.transform(dicts)
y_pred = model.predict(X_val)

In [35]:
df_result = pd.DataFrame()
df_result['ride_id'] = df['ride_id']
df_result['predicted_duration'] = y_pred
df_result.to_parquet(
    'qwerty.parquet',
    engine='pyarrow',
    compression=None,
    index=False
)